In [39]:
import os 
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec

load_dotenv()

True

In [40]:

try:
    pc = Pinecone(api_key = os.getenv('PINECONE_API_KEY'))
    pc.create_index(
        name = 'rag',
        dimension = 384,
        metric = 'cosine',
        spec = ServerlessSpec(
            cloud = 'aws',
            region = 'us-east-1'
        )
    )
except:
    pass 

In [41]:
import json 
data = json.load(open('reviews.json'))
data['reviews']

[{'professor': 'Dr. John Smith',
  'subject': 'Computer Science',
  'stars': 4,
  'review': 'Great professor, explains concepts clearly and is very approachable.'},
 {'professor': 'Prof. Emily Johnson',
  'subject': 'Mathematics',
  'stars': 5,
  'review': 'Amazing teacher! Her lectures are well-organized and easy to follow.'},
 {'professor': 'Dr. Michael Brown',
  'subject': 'Physics',
  'stars': 3,
  'review': 'The class was tough, but he knows his stuff. Be prepared to study hard.'},
 {'professor': 'Prof. Sarah Lee',
  'subject': 'Biology',
  'stars': 2,
  'review': 'Lectures can be boring, and the material is not well-organized.'},
 {'professor': 'Dr. David Wilson',
  'subject': 'Chemistry',
  'stars': 5,
  'review': 'Very enthusiastic about the subject and makes the lectures interesting.'},
 {'professor': 'Prof. Linda Davis',
  'subject': 'History',
  'stars': 4,
  'review': 'Good professor, but the exams are a bit challenging.'},
 {'professor': 'Dr. Karen Martinez',
  'subject': 

In [42]:
from sentence_transformers import SentenceTransformer

processed_data = [] 
for review in data['reviews']:
    model = SentenceTransformer("all-MiniLM-L6-v2")
    sentence = review['review']
    
    embeddings = model.encode(sentence)
    print(embeddings.shape)
    processed_data.append( {
        'values': embeddings,
        'id': review['professor'],
        'metadata': {
            'review': review['review'],
             'subject': review['subject'],
             'stars' : review['stars']  
        }
    })

(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)
(384,)


In [43]:
index = pc.Index('rag')
index.upsert(
    vectors = processed_data,
    namespace = "ns1"
)

{'upserted_count': 20}

In [44]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}